# Solving the n-Queens Problem using Local Search

Student Name: [Add your name]

I have used the following AI tools: [list tools]

I understand that my submission needs to be my own work: [your initials]


## Learning Outcomes

* Implement multiple hill climbing search variants to solve the n-Queens problem.
* Apply simulated annealing with appropriate temperature scheduling to overcome local optima.
* Compare algorithm performance using runtime, solution quality, and success rate metrics.
* Analyze and visualize algorithm performance across different problem sizes.
* Graduate Students: Design and test alternative local move operators to improve search efficiency.

## Instructions

Total Points: Undergrads 100 + 5 bonus / Graduate students 110

Complete this notebook. Use the provided notebook cells and insert additional code and markdown cells as needed. Submit the completely rendered notebook as a HTML file. 

## The n-Queens Problem

* __Goal:__ Find an arrangement of $n$ queens on a $n \times n$ chess board so that no queen is on the same row, column or diagonal as any other queen.

* __State space:__ An arrangement of the queens on the board. We restrict the state space to arrangements where there is only a single queen per column. We represent a state as an integer vector $\mathbf{q} = \{q_1, q_2, \dots, q_n\}$, each number representing the row positions of the queens from left to right. We will call a state a "board."

* __Objective function:__ The number of pairwise conflicts (i.e., two queens in the same row/column/diagonal).
The optimization problem is to find the optimal arrangement $\mathbf{q}^*$ of $n$ queens on the board can be written as:

  > minimize: $\mathrm{conflicts}(\mathbf{q})$
  >
  > subject to: $\mathbf{q} \ \text{contains only one queen per column}$

  Note: the constraint (subject to) is enforced by the definition of the state space.

* __Local improvement move:__ Move one queen to a different row in its column.

* __Termination:__ For this problem there is always an arrangement $\mathbf{q}^*$ with $\mathrm{conflicts}(\mathbf{q}^*) = 0$, however, the local improvement moves might end up in a local minimum. 

## Helper functions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors

np.random.seed(1234)


def random_board(n):
    """Tạo một bàn cờ ngẫu nhiên kích thước n x n.
    Lưu ý: chỉ đặt một quân ở mỗi cột (mỗi cột có một giá trị hàng).
    Trả về một mảng số nguyên độ dài n, mỗi phần tử là vị trí hàng của quân ở cột tương ứng.
    """
    
    return(np.random.randint(0,n, size = n))

def comb2(n): return n*(n-1)//2 # tổ hợp 2 (n choose 2)

def conflicts(board):
    """Tính số xung đột (objective function).
    Sử dụng đếm số quân ở cùng hàng và cùng hai đường chéo.
    Trả về tổng các cặp xung đột (số nguyên >= 0).
    """

    n = len(board)
    
    horizontal_cnt = [0] * n
    diagonal1_cnt = [0] * 2 * n
    diagonal2_cnt = [0] * 2 * n
    
    for i in range(n):
        horizontal_cnt[board[i]] += 1
        diagonal1_cnt[i + board[i]] += 1
        diagonal2_cnt[i - board[i] + n] += 1
    
    return sum(map(comb2, horizontal_cnt + diagonal1_cnt + diagonal2_cnt))

# giảm kích thước font để hiển thị các bàn lớn hơn
def show_board(board, cols = ['white', 'gray'], fontsize = 48):  
    """Hiển thị bàn cờ và các quân hậu lên đồ thị.
    - board: mảng chứa vị trí hàng của hậu theo cột (0-indexed)
    - cols: màu cho ô cờ
    - fontsize: kích thước ký hiệu quân hậu
    """
    
    n = len(board)
    
    # tạo ma trận hiển thị dạng bàn cờ
    display = np.zeros([n,n])
    for i in range(n):
        for j in range(n):
            if (((i+j) % 2) != 0): 
                display[i,j] = 1
    
    cmap = colors.ListedColormap(cols)
    fig, ax = plt.subplots()
    ax.imshow(display, cmap = cmap, 
              norm = colors.BoundaryNorm(range(len(cols)+1), cmap.N))
    ax.set_xticks([])
    ax.set_yticks([])
    
    # đặt ký hiệu hậu bằng Unicode u265B
    for j in range(n):
        plt.text(j, board[j], u"\u265B", fontsize = fontsize, 
                 horizontalalignment = 'center',
                 verticalalignment = 'center')
    
    print(f"Bàn với {conflicts(board)} xung đột.")
    plt.show()

## Create a board

In [ ]:
# Tạo một bàn cờ ngẫu nhiên và hiển thị (ví dụ)
board = random_board(4)

show_board(board)
print(f"Vị trí các hậu (từ trái sang phải): {board}")
print(f"Số xung đột: {conflicts(board)}")

A board $4 \times 4$ with no conflicts:

In [ ]:
board = [1,3,0,2]
show_board(board)

# Tasks

## General [10 Points]

1. Make sure that you use the latest version of this notebook. Sync your forked repository and pull the latest revision. 
2. Your implementation can use libraries like math, numpy, scipy, but not libraries that implement intelligent agents or complete search algorithms. Try to keep the code simple! In this course, we want to learn about the algorithms and we often do not need to use object-oriented design.
3. You notebook needs to be formatted professionally. 
    - Add additional markdown blocks for your description, comments in the code, add tables and use mathplotlib to produce charts where appropriate
    - Do not show debugging output or include an excessive amount of output.
    - Check that your submitted file is readable and contains all figures.
4. Document your code. Use comments in the code and add a discussion of how your implementation works and your design choices.

## Task 1: Steepest-ascend Hill Climbing Search [20 Points]

Calculate the objective function for all local moves (see definition of local moves above) and always choose the best among all local moves. If there are no local moves that improve the objective, then you have reached a local optimum. 

In [ ]:
# Task 1: Steepest-ascent Hill Climbing (Tìm kiếm leo đồi theo hướng dốc lớn nhất)
import time
import random


def steepest_ascent_hill_climbing(start_board, max_iters=1000):
    """Thuật toán steepest-ascent hill climbing.
    Ở mỗi bước, đánh giá tất cả các di chuyển thay đổi vị trí 1 quân trong một cột
    và chọn di chuyển cải thiện nhất (giảm số xung đột nhiều nhất).

    Trả về: final_board (np.array), history (danh sách số xung đột theo thời gian)
    """
    board = np.array(start_board).copy()
    n = len(board)
    history = [conflicts(board)]
    it = 0
    while it < max_iters:
        it += 1
        current_conf = conflicts(board)
        best_conf = current_conf
        best_board = None
        # đánh giá tất cả di chuyển đơn quân
        for col in range(n):
            orig = board[col]
            for row in range(n):
                if row == orig:
                    continue
                board[col] = row
                c = conflicts(board)
                if c < best_conf:
                    best_conf = c
                    best_board = board.copy()
            board[col] = orig
        # Nếu không tìm được cải thiện, đã tới cực trị cục bộ
        if best_board is None:
            break
        board = best_board
        history.append(best_conf)
        if best_conf == 0:
            break
    return board, history

# Kiểm tra nhanh (ví dụ nhỏ)
if __name__ == '__main__':
    b = random_board(8)
    show_board(b)
    final, hist = steepest_ascent_hill_climbing(b)
    print('Xung đột ban đầu:', conflicts(b), 'Xung đột cuối:', conflicts(final))

## Task 2: Stochastic Hill Climbing 1 [10 Points]

Chooses randomly from among all uphill moves till you have reached a local optimum.

In [ ]:
# Task 2: Stochastic Hill Climbing 1
# Chọn ngẫu nhiên trong số tất cả các di chuyển cải thiện

def stochastic_hill_climbing_all_neighbors(start_board, max_iters=1000):
    """Thuật toán stochastic HC 1:
    Ở mỗi bước, thu thập tất cả các di chuyển cải thiện và chọn ngẫu nhiên một trong số đó.
    Trả về board cuối cùng và lịch sử số xung đột.
    """
    board = np.array(start_board).copy()
    n = len(board)
    history = [conflicts(board)]
    it = 0
    while it < max_iters:
        it += 1
        current_conf = conflicts(board)
        improving_moves = []  # danh sách tuple (conf, col, row)
        for col in range(n):
            orig = board[col]
            for row in range(n):
                if row == orig:
                    continue
                board[col] = row
                c = conflicts(board)
                if c < current_conf:
                    improving_moves.append((c, col, row))
            board[col] = orig
        if not improving_moves:
            break
        # chọn ngẫu nhiên trong số các di chuyển cải thiện
        c, col, row = random.choice(improving_moves)
        board[col] = row
        history.append(c)
        if c == 0:
            break
    return board, history

# kiểm tra nhanh
if __name__ == '__main__':
    b = random_board(8)
    show_board(b)
    final, hist = stochastic_hill_climbing_all_neighbors(b)
    print('Xung đột ban đầu:', conflicts(b), 'Xung đột cuối:', conflicts(final))


## Task 3: Stochastic Hill Climbing 2 [20 Points]

A popular version of stochastic hill climbing generates only a single random local neighbor at a time and accept it if it has a better objective function value than the current state. This is very efficient if each state has many possible successor states. This method is called "First-choice hill climbing" in the textbook.

__Notes:__ 

* Detecting local optima is tricky! You can, for example, stop if you were not able to improve the objective function during the last $x$ tries.

In [ ]:
# Task 3: Stochastic Hill Climbing 2 (First-choice hill climbing)
# Sinh một láng giềng ngẫu nhiên một lần và chấp nhận nếu tốt hơn

def first_choice_hill_climbing(start_board, max_iters=10000, no_improve_limit=1000):
    """First-choice hill climbing:
    Sinh ngẫu nhiên một láng giềng (chọn cột và hàng mới) và chấp nhận ngay nếu nó cải thiện.
    Dừng nếu không cải thiện trong một số lượng thử liên tiếp (no_improve_limit).
    """
    board = np.array(start_board).copy()
    n = len(board)
    history = [conflicts(board)]
    it = 0
    no_improve = 0
    current_conf = conflicts(board)
    while it < max_iters and no_improve < no_improve_limit:
        it += 1
        # sinh một láng giềng ngẫu nhiên
        col = random.randrange(n)
        orig = board[col]
        row = random.randrange(n)
        if row == orig:
            continue
        board[col] = row
        c = conflicts(board)
        if c < current_conf:
            current_conf = c
            history.append(c)
            no_improve = 0
            if c == 0:
                break
        else:
            # hoàn lại nếu không chấp nhận
            board[col] = orig
            no_improve += 1
    return board, history

# kiểm tra nhanh
if __name__ == '__main__':
    b = random_board(8)
    show_board(b)
    final, hist = first_choice_hill_climbing(b)
    print('Xung đột ban đầu:', conflicts(b), 'Xung đột cuối:', conflicts(final))


## Task 4: Hill Climbing Search with Random Restarts [10 Points]

Hill climbing will often end up in local optima. Restart the each of the three hill climbing algorithm up to 100 times with a random board to find a better (hopefully optimal) solution. Note that restart just means to run the algorithm several times starting with a new random board.

In [ ]:
# Task 4: Khởi động lại ngẫu nhiên (random restarts)
# Hàm hỗ trợ chạy thuật toán nhiều lần với các bàn khởi tạo khác nhau và thu thập kết quả

def random_restarts(algorithm_fn, n, restarts=100, **kwargs):
    results = []
    times = []
    for i in range(restarts):
        start = random_board(n)
        t0 = time.time()
        final, hist = algorithm_fn(start, **kwargs)
        t1 = time.time()
        results.append({'start_conf': conflicts(start), 'end_conf': conflicts(final), 'history': hist})
        times.append(t1 - t0)
    return results, times

# Ví dụ kiểm tra nhanh với n=8 và 20 lần khởi động lại
if __name__ == '__main__':
    res, ts = random_restarts(steepest_ascent_hill_climbing, 8, restarts=20)
    print('Xung đột tối thiểu sau các lần chạy:', min(r['end_conf'] for r in res), 'Thời gian trung bình (s):', np.mean(ts))

## Task 5: Simulated Annealing [10 Points]

Simulated annealing is a form of stochastic hill climbing that avoid local optima by also allowing downhill moves with a probability proportional to a temperature. The temperature is decreased in every iteration following an annealing schedule. You have to experiment with the annealing schedule (Google to find guidance on this).


1. Implement simulated annealing for the n-Queens problem.
2. Create a visualization of the search process (a line chart of how the number if conflict changes as the algorithm progrsses).
3. Use this visualization for experiments with different choices for the annealing schedule and discuss what you have learned.

In [ ]:
# Task 5: Simulated Annealing (Mô phỏng 'làm nguội'
# Áp dụng lịch làm nguội mũ (exponential cooling) để cho phép chấp nhận bước xấu theo xác suất

def simulated_annealing(start_board, max_iters=10000, t0=1.0, alpha=0.995):
    board = np.array(start_board).copy()
    n = len(board)
    history = [conflicts(board)]
    current_conf = conflicts(board)
    T = t0
    for it in range(max_iters):
        # sinh một láng giềng ngẫu nhiên
        col = random.randrange(n)
        orig = board[col]
        row = random.randrange(n)
        if row == orig:
            continue
        board[col] = row
        c = conflicts(board)
        delta = c - current_conf
        if delta <= 0:
            # chấp nhận nếu cải thiện
            current_conf = c
            history.append(c)
        else:
            # chấp nhận với xác suất exp(-delta/T)
            if T <= 0:
                accept = False
            else:
                accept = (random.random() < np.exp(-delta / T))
            if accept:
                current_conf = c
                history.append(c)
            else:
                board[col] = orig
        # làm nguội
        T *= alpha
        if current_conf == 0:
            break
        if T < 1e-12:
            break
    return board, history

# kiểm tra nhanh
if __name__ == '__main__':
    b = random_board(8)
    show_board(b)
    final, hist = simulated_annealing(b, max_iters=2000)
    print('Xung đột ban đầu:', conflicts(b), 'Xung đột cuối:', conflicts(final))

## Task 6: Algorithm Behavior Analysis [20 Points]

### Comparison
Compare the algorithm using runtime and objective function values. Use boards of size 4 and 8 to explore how the different algorithms perform. Make sure that you run the algorithms for each board size several times (at least 100 times) with different starting boards and report averages.

Complete the following table

| Algorithm           | Board size | Avg. Run time | Avg. number of conflicts | % of runs ending in optimal solution  |
| ------------------- | ---------- | ------------- | --------------------------------- | - |
| Steepest asc. HC    |     4      |               |                                   |   |
| Stochastic HC 1     |     4      |               |                                   |   |
| Stochastic HC 2     |     4      |               |                                   |   |
| Simulated Annealing |     4      |               |                                   |   |
| Steepest asc. HC    |     8      |               |                                   |   |
| Stochastic HC 1     |     8      |               |                                   |   |
| Stochastic HC 2     |     8      |               |                                   |   |
| Simulated Annealing |     8      |               |                                   |   |

Hint: See [Profiling Python Code](../HOWTOs/profiling_code.ipynb) for help about how to measure runtime in Python.

Add the used code here:

---

Discussion (Nhận xét và điều học được):

- Mục tiêu của phần so sánh là tóm tắt hiệu năng thực nghiệm: thời gian chạy trung bình, chất lượng nghiệm (số xung đột trung bình) và tỷ lệ chạy đạt nghiệm tối ưu (0 xung đột).
- Từ các thuật toán đã triển khai thường thu được những quan sát sau:
  - Steepest-ascent HC: thường có cải thiện nhanh ban đầu (vì đánh giá toàn bộ không gian lân cận để chọn bước tốt nhất), nhưng dễ bị mắc kẹt tại cực trị cục bộ. Chi phí mỗi bước cao hơn (do xem xét tất cả các láng giềng), nên thời gian chạy trung bình có thể lớn hơn so với các biến thể mang tính ngẫu nhiên.
  - Stochastic HC 1 (chọn ngẫu nhiên từ các bước cải thiện): cho hiệu năng ổn định hơn so với steepest trong một số trường hợp, vì chọn ngẫu nhiên giữa các bước tốt giúp tránh một vài bẫy cục bộ nhỏ, nhưng vẫn có khả năng bị dừng ở cực trị cục bộ.
  - Stochastic HC 2 / First-choice: thường có bước tiến nhanh hơn (ít phải đánh giá nhiều láng giềng), phù hợp với bài toán có số láng giềng lớn; tuy nhiên phụ thuộc mạnh vào may rủi của mẫu, nên phương pháp này có phương sai lớn hơn về kết quả cuối cùng.
  - Simulated Annealing: chậm hơn từng bước so với first-choice nhưng có khả năng vượt khỏi cực trị cục bộ nhờ chấp nhận các bước xấu với xác suất giảm dần. Trong thực nghiệm, SA thường có tỷ lệ đạt nghiệm tối ưu cao hơn khi lịch làm lạnh và tham số được chỉnh hợp lý.
- Kết luận thực tế: không có thuật toán nào "tốt nhất" cho mọi kích thước; steepest tốt về chất lượng bước nhưng tốn thời gian, first-choice phù hợp khi cần chạy nhanh nhiều lần, và simulated annealing là lựa chọn hợp lý khi muốn ưu tiên tìm nghiệm tối ưu hơn và chấp nhận chi phí tính toán cao hơn.

In [ ]:
# Task 6: Phân tích hành vi thuật toán (so sánh)
import timeit
import pandas as pd

algorithms = [
    ("Steepest", steepest_ascent_hill_climbing),
    ("StochasticAll", stochastic_hill_climbing_all_neighbors),
    ("FirstChoice", first_choice_hill_climbing),
    ("SimulatedAnnealing", simulated_annealing)
]

def evaluate_algorithms(sizes=[4,8], runs=100, restarts=10):
    """Đánh giá các thuật toán theo thời gian chạy trung bình, số xung đột trung bình
    và tỷ lệ chạy đạt nghiệm tối ưu cho các kích thước bàn khác nhau.
    Trả về một DataFrame tóm tắt kết quả.
    """
    rows = []
    for size in sizes:
        for name, fn in algorithms:
            run_times = []
            end_confs = []
            success_count = 0
            for r in range(runs):
                start = random_board(size)
                t0 = time.time()
                final, hist = fn(start)
                t1 = time.time()
                run_times.append(t1 - t0)
                end_confs.append(conflicts(final))
                if conflicts(final) == 0:
                    success_count += 1
            rows.append({
                'Algorithm': name,
                'Board size': size,
                'Avg Run time': np.mean(run_times),
                'Avg conflicts': np.mean(end_confs),
                '% optimal': 100.0 * success_count / runs
            })
    return pd.DataFrame(rows)

# ví dụ chạy nhanh (giảm số lần để demo)
if __name__ == '__main__':
    df = evaluate_algorithms(sizes=[4,8], runs=30)
    display(df)

### Algorithm Convergence (Hội tụ của thuật toán)

Hướng dẫn & yêu cầu (bằng tiếng Việt, rõ ràng):

- Mục tiêu: phân tích hành vi hội tụ của từng thuật toán đã triển khai (Steepest-ascent HC, Stochastic HC 1, First-choice HC, Simulated Annealing).

- Việc cần làm:
  1. Chạy từng thuật toán trên bài toán 8-Queens ít nhất 30 lần với các khởi tạo khác nhau (để có run đại diện). Chọn 1 run tiêu biểu cho mỗi thuật toán để minh họa.
  2. Vẽ đồ thị "số xung đột theo số bước" cho mỗi run tiêu biểu (conflicts vs iteration). Hiển thị tất cả đường trên cùng một biểu đồ để so sánh.
  3. Mô tả bằng tiếng Việt các mẫu hội tụ quan sát được: tốc độ cải thiện ban đầu, plateau (đóng băng), các bước nhảy (jumps) do hành vi ngẫu nhiên, và khi nào SA chấp nhận bước xấu.
  4. Ghi lại các tham số thực nghiệm (số bước tối đa, no_improve_limit, t0, alpha, v.v.) dùng cho các run này.

- Đầu ra mong đợi (nộp kèm trong notebook):
  - Một hoặc hai biểu đồ (representative runs) trên 8-Queens.
  - Một đoạn văn ngắn (3–6 câu) bằng tiếng Việt giải thích mô tả hội tụ cho mỗi thuật toán.

- Chú ý: biểu đồ nên dùng thang tuyến tính (iteration trên trục x, conflicts trên trục y) để nhìn rõ plateau và sự khác biệt ban đầu.

In [ ]:
# Vẽ đồ thị hội tụ đại diện cho các thuật toán trên 8-queens
# Mỗi thuật toán chạy một lần với khởi tạo khác nhau để minh họa xu hướng hội tụ

def plot_representative_runs():
    fig, ax = plt.subplots(figsize=(9,5))
    for name, fn in algorithms:
        start = random_board(8)
        final, hist = fn(start)
        ax.plot(hist, label=name)
    ax.set_xlabel('Số bước (Iteration)')
    ax.set_ylabel('Số xung đột (conflicts)')
    ax.set_title('Mô tả hội tụ: các run đại diện trên 8-Queens')
    ax.grid(True)
    ax.legend()
    plt.tight_layout()
    plt.show()

# Gọi hàm để hiển thị
if __name__ == '__main__':
    plot_representative_runs()

### Problem Size Scalability (Độ phóng to theo kích thước bài toán)

In [ ]:
# Đo độ phóng to theo kích thước bài toán (log-log plot)
# Thận trọng: giảm số lần chạy (runs) nếu quá chậm.

def measure_scaling(sizes=[4,8,12,16], runs=5, algorithms_to_test=None):
    """Trả về dict chứa kích thước và thời gian trung bình cho mỗi thuật toán.
    algorithms_to_test: list các tuple (label, function)
    """
    if algorithms_to_test is None:
        algorithms_to_test = [("Steepest", steepest_ascent_hill_climbing), ("FirstChoice", first_choice_hill_climbing)]

    results = { 'size': list(sizes) }
    for name, _ in algorithms_to_test:
        results[name] = []

    for size in sizes:
        for name, fn in algorithms_to_test:
            times = []
            for r in range(runs):
                start = random_board(size)
                t0 = time.time()
                fn(start)
                t1 = time.time()
                times.append(t1 - t0)
            results[name].append(np.mean(times))
    return results

# Ví dụ chạy nhanh (chỉ dùng 4 kích thước để demo)
if __name__ == '__main__':
    res = measure_scaling(sizes=[4,8,12,16], runs=3)
    plt.loglog(res['size'], res['Steepest'], marker='o', label='Steepest')
    plt.loglog(res['size'], res['FirstChoice'], marker='o', label='FirstChoice')
    plt.xlabel('Kích thước bàn (n)')
    plt.ylabel('Thời gian trung bình (s)')
    plt.title('Độ phóng to thời gian theo kích thước (log-log)')
    plt.legend()
    plt.grid(True)
    plt.show()

### Advanced task: Exploring other Local Moves Operators
- Từ việc thử nghiệm các move operators khác nhau (single-step, column-swap, dual-queen, adaptive) ta rút ra:
  - Column-swap hữu ích khi nhiều cột có cấu trúc xung đột lẫn nhau; việc hoán đổi đôi khi nhanh chóng khôi phục trạng thái không xung đột.
  - Single-step phù hợp khi cần tinh chỉnh nhẹ vị trí của các quân, ít phá vỡ cấu trúc tổng thể.
  - Dual-queen move và các move thay đổi nhiều biến cùng lúc có thể giúp thoát khỏi bẫy cục bộ nhưng cũng khiến quá trình tìm kiếm mất ổn định hơn.
  - Adaptive move (tập trung vào cột có xung đột nhiều) thường cho hiệu quả thực nghiệm tốt vì nó kết hợp khám phá có định hướng và randomization.

- Bài học chung:
  - Không có move operator tốt nhất cho mọi trường hợp; việc kết hợp vài operator và lựa chọn ngẫu nhiên theo trạng thái hiện tại thường cho kết quả khả quan.
  - Việc đo lường và so sánh bằng thực nghiệm là thiết yếu — các giả thuyết lý thuyết chỉ dẫn hướng, nhưng thực nghiệm mới cho biết tham số tối ưu cho từng kích thước và mục tiêu.

In [ ]:
# Các Move Operator nâng cao 
# Triển khai: single-step, column-swap, dual-queen, adaptive

import copy

def single_step_move(board):
    """Di chuyển một quân một ô lên hoặc xuống (wrap-around).
    - board: mảng 1D (numpy array hoặc list) biểu diễn vị trí hàng của quân theo cột.
    Trả về bảng mới (copy) sau di chuyển.
    """
    b = board.copy()
    n = len(b)
    col = random.randrange(n)
    direction = random.choice([-1, 1])
    b[col] = (int(b[col]) + direction) % n
    return b

def column_swap_move(board):
    """Hoán đổi vị trí của hai cột (đổi chỗ 2 quân hậu giữa 2 cột).
    Trả về bảng mới.
    """
    b = board.copy()
    n = len(b)
    c1, c2 = random.sample(range(n), 2)
    b[c1], b[c2] = b[c2], b[c1]
    return b

def dual_queen_move(board):
    """Chọn hai cột và đặt lại vị trí hàng của cả hai quân một cách ngẫu nhiên."""
    b = board.copy()
    n = len(b)
    c1, c2 = random.sample(range(n), 2)
    b[c1] = random.randrange(n)
    b[c2] = random.randrange(n)
    return b

# Hàm phụ: tính số xung đột mà một quân ở cột `col` đang tạo ra
def local_conflicts(board, col):
    """Trả về số lượng quân khác đang xung đột với quân ở cột `col`.
    Dùng để xác định cột nào có vấn đề và nên tập trung.
    """
    n = len(board)
    cnt = 0
    for j in range(n):
        if j == col:
            continue
        # cùng hàng
        if board[j] == board[col]:
            cnt += 1
        # cùng đường chéo
        if abs(board[j] - board[col]) == abs(j - col):
            cnt += 1
    return cnt

class AdaptiveMove:
    """Move adaptive: tập trung vào cột có nhiều xung đột.
    - move(board): trả về board mới sau một move áp dụng chiến lược thích ứng.
    """
    def __init__(self):
        self.history = []

    def move(self, board):
        n = len(board)
        # tìm các cột có xung đột > 0
        conflict_cols = [col for col in range(n) if local_conflicts(board, col) > 0]
        if conflict_cols:
            col = random.choice(conflict_cols)
            new_board = board.copy()
            # thử đặt ngẫu nhiên một hàng mới cho cột chọn
            new_board[col] = random.randrange(n)
            return new_board
        # nếu không có cột nào xung đột, fallback về single step
        return single_step_move(board)

# Sử dụng first-choice hill climbing với operator tuỳ chọn
def first_choice_with_operator(start_board, move_operator, max_iters=10000, no_improve_limit=1000):
    """Phiên bản first-choice hill climbing cho phép truyền move_operator(board)."""
    board = np.array(start_board).copy()
    n = len(board)
    history = [conflicts(board)]
    it = 0
    no_improve = 0
    current_conf = conflicts(board)
    while it < max_iters and no_improve < no_improve_limit:
        it += 1
        candidate = move_operator(board)
        c = conflicts(candidate)
        if c < current_conf:
            board = np.array(candidate)
            current_conf = c
            history.append(c)
            no_improve = 0
            if c == 0:
                break
        else:
            no_improve += 1
    return board, history

# Ví dụ nhanh
if __name__ == '__main__':
    b = random_board(8)
    show_board(b)
    final, hist = first_choice_with_operator(b, column_swap_move)
    print('Xung đột ban đầu:', conflicts(b), 'Xung đột cuối:', conflicts(final))

## More Things to Do (not for credit)

If the assignment was to easy for yuo then you can think about the following problems. These problems are challenging and not part of this assignment. 

### Implement a Genetic Algorithm for the n-Queens problem

In [ ]:
# Ví dụ: Runner kiểm thử nhanh cho các thuật toán (in kết quả tóm tắt)
# Chạy các thuật toán trên kích thước nhỏ và in bảng tóm tắt nhanh

def quick_demo(runs=10):
    algs = [
        ("Steepest", steepest_ascent_hill_climbing),
        ("StochasticAll", stochastic_hill_climbing_all_neighbors),
        ("FirstChoice", first_choice_hill_climbing),
        ("SA", simulated_annealing)
    ]
    rows = []
    for name, fn in algs:
        times = []
        end_conf = []
        for i in range(runs):
            start = random_board(8)
            t0 = time.time()
            final, hist = fn(start)
            t1 = time.time()
            times.append(t1 - t0)
            end_conf.append(conflicts(final))
        rows.append((name, np.mean(times), np.mean(end_conf), 100.0 * sum(1 for c in end_conf if c==0)/runs))
    # Hiển thị
    print(f"{'Alg':15s} {'AvgTime(s)':>10s} {'AvgConf':>10s} {'%Optimal':>10s}")
    for r in rows:
        print(f"{r[0]:15s} {r[1]:10.4f} {r[2]:10.2f} {r[3]:10.1f}")

# Gọi demo nhanh nếu chạy file
if __name__ == '__main__':
    quick_demo(runs=10)